# Set up environment

In [ ]:
import sys
print(sys.prefix)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import urllib, os,sys, pdfplumber, glob, requests, wordcloud, re, dateparser

# Set up working dir

In [ ]:
base_dir = os.path.realpath('../..')
print(base_dir)
data_dir = base_dir + '\Data'

## Load speaking turns data

In [ ]:
in_dir = data_dir + '\\TK_commissieVWS\\auto_download_20230118'

In [ ]:
all_speaking_turns = pd.read_csv(in_dir + '\speaking_turns_clean_coded_noSEGV.csv', header = 0, index_col = 0)

In [ ]:
all_speaking_turns.shape

In [ ]:
all_speaking_turns.head()

In [ ]:
all_speaking_turns['LEEF'].sum()

## Label data with predictors: LR, GALTAN

In [ ]:
pol_orient = pd.read_excel(base_dir + '\\Data\\KK 2006-2021 coordinaten partijen nieuw.xlsx', sheet_name = 'cleaned')
pol_orient['party'] = pol_orient['party'].apply(lambda x: x.lower())

In [ ]:
pol_orient

##### Add in missing parties

In [ ]:
parties_with_orient = pol_orient['party'].unique().tolist()
parties_with_orient = np.sort(parties_with_orient)
print(parties_with_orient)

In [ ]:
parties_with_speaking_turn = all_speaking_turns['party'].unique().tolist()
# print(parties_with_speaking_turn)

In [ ]:
parties_without_orient = [a for a in parties_with_speaking_turn if a not in parties_with_orient]
parties_without_orient = np.sort(parties_without_orient)
print(parties_without_orient)

In [ ]:
party_orient_copy_dict = {
    'bontes-klaveren':'pvv',
    'den haan':'50plus',
    'gkvk':'50plus',
    'kuzu-öztürk':'denk',
    'klein':'50plus',
    'krol':'50plus',
    'leijten':'sp',
    'lid omtzigt':'cda',
    'monasch':'pvda',
    'van haga':'fvd',
    'verdonk':'vvd',
    'vka':'pvdd'
}

In [ ]:
for oldname in party_orient_copy_dict:
    newname = party_orient_copy_dict[oldname]
    print(oldname)
    tmp = pol_orient.loc[pol_orient['party']==newname,:].copy()
    tmp['party'] = oldname
    pol_orient = pol_orient.append(tmp)

In [ ]:
pol_orient = pol_orient.sort_values(['year','party'])
parties_with_orient = pol_orient['party'].unique().tolist()
print(parties_with_orient)

In [ ]:
pol_orient

##### Interpolate years

In [ ]:
years = pd.DataFrame(np.arange(2008,2023), columns = ['year'])

In [ ]:
pol_orient_interpolate = pd.DataFrame()
for pi,party in enumerate(pol_orient['party'].unique()):
    print(party)
    tmp = pol_orient.loc[pol_orient['party']==party].copy().reset_index(drop=True)
    tmp = tmp.merge(years, on = 'year', how = 'outer').sort_values('year')
    tmp = tmp.interpolate(method = 'linear', limit_direction ='forward').interpolate(method = 'linear', limit_direction ='backward')
    tmp['party'] = party
    pol_orient_interpolate = pol_orient_interpolate.append(tmp)

In [ ]:
pol_orient_interpolate.to_csv(base_dir + '\\Data\\KiesKompas_interpolated.csv')

##### Merge into speaking turns data

In [ ]:
vws_data = all_speaking_turns.merge(pol_orient_interpolate[['party','LR','TANGAL','year']], on = ['party','year'])[
    ['year','date','doc_nr','turn_nr','gender','name','party','LR','TANGAL','text','LEEF','SDOH']]

In [ ]:
vws_data['class'] = 'other'
vws_data.loc[vws_data['LEEF']==1,'class'] = 'LEEF'
vws_data.loc[vws_data['SDOH']==1,'class'] = 'SDOH'
vws_data.loc[((vws_data['LEEF']==1) & (vws_data['SDOH']==1)),'class'] = 'both'

In [ ]:
vws_data.head()

In [ ]:
vws_data['LEEF'].sum()

## Label data with predictors: cabinet ID and time since installation

In [ ]:
from datetime import datetime
date_format = "%Y-%m-%d"

In [ ]:
cabinets = pd.DataFrame({
    'Balkenende-IV':'2006-11-30',
    'Rutte-I':'2010-06-17',
    'Rutte-II':'2012-09-20',
    'Rutte-III':'2017-03-23',
    'Rutte-IV':'2021-03-31'
}
    , index = [0]).T.reset_index()
cabinets.columns = ['name','date']
cabinets

In [ ]:
for cabi,cabdat in cabinets.iterrows():
    name, date = cabdat.values.tolist()
    if cabi < (cabinets.shape[0] - 1):
        nextdate = cabinets.iloc[cabi+1]['date']
    else:
        nextdate = '2030-01-01'
    print(name,date)
    vws_data.loc[(vws_data['date'] >= date) & (vws_data['date'] < nextdate), 'cabinet'] = name
    dates = vws_data.loc[(vws_data['date'] >= date) & (vws_data['date'] < nextdate), 'date'].apply(lambda x: datetime.strptime(x, date_format))
    days_diffs = (dates - datetime.strptime(date, date_format)).apply(lambda x: x.days).values
    vws_data.loc[(vws_data['date'] >= date) & (vws_data['date'] < nextdate), 'days_since_installation'] = days_diffs
    vws_data.loc[(vws_data['date'] >= date) & (vws_data['date'] < nextdate), 'month_since_installation'] = (np.ceil(np.divide(days_diffs,(365.25/12)))).astype(int)
    vws_data.loc[(vws_data['date'] >= date) & (vws_data['date'] < nextdate), 'share_of_period'] = (days_diffs - np.min(days_diffs))/np.max(days_diffs - np.min(days_diffs))